In [1]:
from IPython.display import display
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
alpha, Delta, Delta_star, sigma, n, T, vT, g, m, phi = sp.symbols('alpha Delta Delta^* sigma n T v_T chi m phi', real = True, positive=True)

In [2]:
class CollisionIntegrator:
    def __init__(self):
        self.Cx, self.Cy, self.s, self.psi = sp.symbols('Cx Cy s psi', real = True, positive = True)
        self.pi = sp.pi

 
        self.hat_sigma = sp.Matrix([sp.cos(self.psi), sp.sin(self.psi)])
        self.sigma_perp = sp.Matrix([-sp.sin(self.psi), sp.cos(self.psi)])

        # Center coordinates and relative vector
        self.c12 = sp.Matrix([self.s, 0])
        self.C = sp.Matrix([self.Cx, self.Cy])
        self.c1 = self.C + sp.Rational(1, 2)*self.c12
        self.c2 = self.C - sp.Rational(1, 2)*self.c12
        self.one_plus = sp.Rational(1, 2)*(1 + alpha)

        # Post-collision (primed) velocities
        self.c1p = self.C + sp.Rational(1, 2)*self.c12 - self.one_plus * (self.s*sp.cos(self.psi))*self.hat_sigma - Delta_star*self.sigma_perp
        self.c2p = self.C - sp.Rational(1, 2)*self.c12 + self.one_plus * (self.s*sp.cos(self.psi))*self.hat_sigma + Delta_star*self.sigma_perp

        # Perpendicular of c2
        self.c2perp = sp.Matrix([-self.c2[1], self.c2[0]])
        self.c1perp = sp.Matrix([-self.c1[1], self.c1[0]])
        
        self.c1p_perp = sp.Matrix([-self.c1p[1], self.c1p[0]])
        self.c2p_perp = sp.Matrix([-self.c2p[1], self.c2p[0]])

        self.psi_bounds = (sp.pi/2, 3*sp.pi/2) 
        self.kernel = -self.s*sp.cos(self.psi)*sp.exp(-self.s**2/2)

    @staticmethod
    def dot(a, b):
        return sp.simplify(a.dot(b))

    @staticmethod
    def norm2(v):
        return sp.simplify(v.dot(v))

    def moment_1d(self, p):
        """\int_{-inf}^{\inf} x^p e^{-ax^2} dx."""
        p = int(p)
        if p%2 == 1:
            return sp.Integer(0)
        k = p//2
        double_fact = sp.factorial2(2*k - 1) if k > 0 else 1
        return sp.sqrt(sp.pi)*double_fact/(2**k*2**(k + sp.Rational(1, 2)))

    def integrate_over_C(self, expr, include_normalization=True):
        """Integrate expr over (Cx, Cy) with weight e^{-a(Cx^2+Cy^2)} via Gaussian moments."""
        expr = sp.expand(expr)
        try:
            poly = sp.Poly(expr, self.Cx, self.Cy)
        except Exception as e:
            raise ValueError("Expression must be polynomial in Cx and Cy.") from e

        I = sp.Integer(0)
        for (px, py), coeff in zip(poly.monoms(), poly.coeffs()):
            mpx = self.moment_1d(px)
            mpy = self.moment_1d(py)
            I += sp.simplify(coeff*mpx * mpy)
        if include_normalization:
            I = sp.simplify(I*(1/self.pi**2))
        return I

    def integrate_over_psi(self, expr):
        lb, ub = self.psi_bounds
        return sp.simplify(sp.integrate(expr, (self.psi, lb, ub)))

    def integrate_over_s(self, expr):
        return sp.simplify(sp.integrate(expr, (self.s, 0, sp.oo)))

    def compute(self, expr):
        """Compute full integral given an integrand expr(Cx,Cy,s,psi,alpha,Delta).
        Steps: C-integration (Gaussian moments) → psi → s, with defaults matching the notebook.
        """
        I_C = self.integrate_over_C(expr)
        integrand = I_C*self.kernel*self.s
        I_psi = self.integrate_over_psi(integrand)
        I_s = self.integrate_over_s(I_psi)
        
        I_s = sp.simplify(2 * self.pi * I_s)  # 2*pi from c12 angle
        return sp.simplify(I_s)
    
    @staticmethod
    def dimensionalize(expr, v_num = 0, m_num = 0):
        factor = g*sigma*n**2*vT**(1 + v_num)*m**m_num
        return sp.simplify(factor * expr)
    
    @staticmethod
    def substitution_delta_star(expr):
        subs = {Delta_star: Delta/vT}
        return expr.subs(subs)
    
    @staticmethod
    def substitution_vt_T(expr):
        subs = {vT: sp.sqrt(2*T/m)}
        return expr.subs(subs)
    
    @staticmethod
    def substitution_Temp(expr):
        subs = {T: m*Delta**2/(1-alpha**2)}
        return expr.subs(subs)
    
    @staticmethod
    def substitution_density(expr):
        subs = {n: 4*phi/(sp.pi*sigma**2)}
        return expr.subs(subs)
I = CollisionIntegrator()


# Steady state temperature

In [3]:
deltaE = sp.simplify(I.norm2(I.c1p) - I.norm2(I.c1) + I.norm2(I.c2p) - I.norm2(I.c2))
deltaE_averaged = I.substitution_vt_T(I.substitution_delta_star(I.compute(deltaE)))
display(deltaE_averaged)
display(sp.Eq(T, sp.solve(deltaE_averaged, T)[0]))

sqrt(2)*sqrt(pi)*(Delta**2*m/T + alpha**2 - 1)

Eq(T, -Delta**2*m/(alpha**2 - 1))

# Frequency of collision

In [4]:
frequency_of_collision = I.substitution_density(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(I.compute(1/n)))))
display(sp.Eq(sp.Symbol("\omega"), frequency_of_collision))

Eq(\omega, 8*sqrt(T)*chi*phi/(sqrt(pi)*sqrt(m)*sigma))

# Pressure and torque density

In [5]:
I_12_diagonal = -(1 + alpha)*(I.dot(I.c1, I.hat_sigma) - I.dot(I.c2, I.hat_sigma))*m*vT/2
virial_pressure = I.compute(I_12_diagonal)

# first 2 comes from overcounting collisions
# second 2 comes from sum_{i,j} = sum_{i<j} + sum_{j<i}
virial_pressure = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(virial_pressure)/2*sigma))/2) 
display(sp.Eq(sp.Symbol("P_{vir}"), virial_pressure))

Eq(P_{vir}, pi*T*chi*n**2*sigma**2*(alpha + 1)/4)

In [6]:
I_12_offdiagonal = m*Delta
torque_density = I.compute(I_12_offdiagonal)
torque_density = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(torque_density)/2*sigma))/2) 
display(sp.Eq(sp.Symbol(r"\tau"), torque_density))

Eq(\tau, sqrt(pi)*Delta*sqrt(T)*chi*sqrt(m)*n**2*sigma**2/2)

# Viscosity

In [7]:

delta_c12_sq = sp.simplify(I.norm2(I.c1p) - I.norm2(I.c1) + I.norm2(I.c2p) - I.norm2(I.c2))
L_parallel_parallel = sp.simplify(
    (I.dot(I.c1p, I.c2))**2 + (I.dot(I.c2p, I.c2))**2
  - (I.dot(I.c1, I.c2))**2  - (I.dot(I.c2, I.c2))**2
  - sp.Rational(1,2) * I.dot(I.c2, I.c2) * delta_c12_sq
)*m**2*vT**4

L_parallel_parallel_averaged = I.compute(L_parallel_parallel)
L_parallel_parallel_averaged = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(L_parallel_parallel_averaged))))



L_parallel_perp = sp.expand(
    I.dot(I.c2perp, I.c1p) * I.dot(I.c2, I.c1p)
  + I.dot(I.c2perp, I.c2p) * I.dot(I.c2, I.c2p)
  - I.dot(I.c2perp, I.c1)  * I.dot(I.c2, I.c1)
)*m**2*vT**4

L_parallel_perp_averaged = I.compute(L_parallel_perp)
L_parallel_perp_averaged = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(L_parallel_perp_averaged))))
display(sp.Eq(sp.Symbol('L_{\\parallel\\parallel}', real=True), L_parallel_parallel_averaged))
display(sp.Eq(sp.Symbol('L_{\\parallel\\parallel}', real = True), I.substitution_Temp(L_parallel_parallel_averaged)).simplify())
display(sp.Eq(sp.Symbol('L_{\\parallel\\perp}', real=True), L_parallel_perp_averaged))
display(sp.Eq(sp.Symbol('L_{\\parallel\\perp}', real = True), I.substitution_Temp(L_parallel_perp_averaged)).simplify())


Eq(L_{\parallel\parallel}, -sqrt(pi)*T**(3/2)*chi*n**2*sigma*(Delta**2*m - 3*T*alpha**2 + 4*T*alpha + 7*T)/sqrt(m))

Eq(L_{\parallel\parallel}, 4*sqrt(pi)*Delta**5*chi*m**2*n**2*sigma*(-1/(alpha**2 - 1))**(5/2)*(alpha**2 - alpha - 2))

Eq(L_{\parallel\perp}, -2*pi*Delta*T**2*chi*n**2*sigma*(alpha - 1))

Eq(L_{\parallel\perp}, -2*pi*Delta**5*chi*m**2*n**2*sigma*(alpha - 1)/(alpha**2 - 1)**2)

In [8]:

A = sp.Matrix([[L_parallel_parallel_averaged, L_parallel_perp_averaged], [-L_parallel_perp_averaged, L_parallel_parallel_averaged]])
D   = 4*n*T*L_parallel_parallel_averaged/(L_parallel_parallel_averaged**2 + L_parallel_perp_averaged**2)
D_o = 4*n*T*L_parallel_perp_averaged/(L_parallel_parallel_averaged**2 + L_parallel_perp_averaged**2)
eta_s = -D*n*T**2
eta_o = D_o*n*T**2
display(sp.Eq(sp.Symbol(r'\eta_s', real=True), eta_s))
display(sp.Eq(sp.Symbol(r'\eta_s', real = True), I.substitution_Temp(eta_s)).simplify())
display(sp.Eq(sp.Symbol(r'\eta_o', real=True), eta_o))
display(sp.Eq(sp.Symbol(r'\eta_o', real = True), I.substitution_Temp(eta_o)).simplify())    

Eq(\eta_s, 4*sqrt(pi)*T**(9/2)*chi*n**4*sigma*(Delta**2*m - 3*T*alpha**2 + 4*T*alpha + 7*T)/(sqrt(m)*(4*pi**2*Delta**2*T**4*chi**2*n**4*sigma**2*(alpha - 1)**2 + pi*T**3*chi**2*n**4*sigma**2*(Delta**2*m - 3*T*alpha**2 + 4*T*alpha + 7*T)**2/m)))

Eq(\eta_s, 4*Delta*m*(-1/(alpha**2 - 1))**(9/2)*(alpha**2 - 1)**4*(alpha**2 - alpha - 2)/(sqrt(pi)*chi*sigma*(pi*(alpha - 1)**2*(alpha**2 - 1) - 4*(alpha**2 - alpha - 2)**2)))

Eq(\eta_o, -8*pi*Delta*T**5*chi*n**4*sigma*(alpha - 1)/(4*pi**2*Delta**2*T**4*chi**2*n**4*sigma**2*(alpha - 1)**2 + pi*T**3*chi**2*n**4*sigma**2*(Delta**2*m - 3*T*alpha**2 + 4*T*alpha + 7*T)**2/m))

Eq(\eta_o, 2*Delta*m*(alpha - 1)/(chi*sigma*(pi*(alpha - 1)**2*(alpha**2 - 1) - 4*(alpha**2 - alpha - 2)**2)))

# Thermal Conductivity

In [9]:
a1  = sp.Rational(1,2) * m * (vT**2) * I.norm2(I.c1)  - 2*T
a2  = sp.Rational(1,2) * m * (vT**2) * I.norm2(I.c2)  - 2*T
a1p = sp.Rational(1,2) * m * (vT**2) * I.norm2(I.c1p) - 2*T
a2p = sp.Rational(1,2) * m * (vT**2) * I.norm2(I.c2p) - 2*T

M_parallell_parallel = sp.simplify(
    a2 * (
        a1p * I.dot(I.c1p, I.c2)*(vT**2)
      + a2p * I.dot(I.c2p, I.c2)*(vT**2)
      - a1  * I.dot(I.c1,  I.c2)*(vT**2)
      - a2  * I.dot(I.c2,  I.c2)*(vT**2)
    )
)

M_parallel_parallel_averaged = I.compute(M_parallell_parallel)

M_parallel_parallel_averaged = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(M_parallel_parallel_averaged))))


M_parallel_perp = sp.simplify(
    a2 * (
        a1p * I.dot(I.c1p, I.c2perp)*(vT**2)
      + a2p * I.dot(I.c2p, I.c2perp)*(vT**2)
      - a1  * I.dot(I.c1,  I.c2perp)*(vT**2)
      - a2  * I.dot(I.c2,  I.c2perp)*(vT**2)
    )
)

M_parallel_perp_averaged = I.compute(M_parallel_perp)
M_parallel_perp_averaged = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(M_parallel_perp_averaged))))

display(sp.Eq(sp.Symbol("M_{\\parallel\\perp}", real=True), M_parallel_perp_averaged))
display(sp.Eq(sp.Symbol("M_{\\parallel\\perp}", real = True), I.substitution_Temp(M_parallel_perp_averaged)).simplify())
display(sp.Eq(sp.Symbol("M_{\\parallel\\parallel}", real=True), M_parallel_parallel_averaged))
display(sp.Eq(sp.Symbol("M_{\\parallel\\parallel}", real = True), I.substitution_Temp(M_parallel_parallel_averaged)).simplify())

Eq(M_{\parallel\perp}, -pi*Delta*T**3*chi*n**2*sigma*(alpha - 1)/m)

Eq(M_{\parallel\perp}, pi*Delta**7*chi*m**2*n**2*sigma*(alpha - 1)/(alpha**2 - 1)**3)

Eq(M_{\parallel\parallel}, sqrt(pi)*T**(5/2)*chi*n**2*sigma*(3*Delta**2*m + 15*T*alpha**2 - 4*T*alpha - 19*T)/(2*m**(3/2)))

Eq(M_{\parallel\parallel}, 2*sqrt(pi)*Delta**7*chi*m**2*n**2*sigma*(-1/(alpha**2 - 1))**(5/2)*(4 - 3*alpha)/(alpha - 1))

In [10]:
Ao = 4*n*T**2/m*M_parallel_parallel_averaged/(M_parallel_perp_averaged**2 + M_parallel_parallel_averaged**2)
Aoperp= 4*n*T**2/m*L_parallel_perp_averaged/(M_parallel_perp_averaged**2 + M_parallel_parallel_averaged**2)
kappa = -2*n*T**2/m*Ao
kappao = 2*n*T**2/m*Aoperp

display(sp.Eq(sp.symbols('\kappa'), kappa))
display(sp.Eq(sp.symbols('\kappa'), I.substitution_Temp(kappa).simplify()))
#display(sp.Eq(sp.symbols('\kappa'), sp.limit(sp.limit(kappa, Delta, 0), alpha, 1)))
display(sp.Eq(sp.symbols('\kappa_o'), kappao))
display(sp.Eq(sp.symbols('\kappa_o'), I.substitution_Temp(kappao).simplify()))

Eq(\kappa, -4*sqrt(pi)*T**(13/2)*chi*n**4*sigma*(3*Delta**2*m + 15*T*alpha**2 - 4*T*alpha - 19*T)/(m**(7/2)*(pi**2*Delta**2*T**6*chi**2*n**4*sigma**2*(alpha - 1)**2/m**2 + pi*T**5*chi**2*n**4*sigma**2*(3*Delta**2*m + 15*T*alpha**2 - 4*T*alpha - 19*T)**2/(4*m**3))))

Eq(\kappa, -16*Delta*(-1/(alpha**2 - 1))**(13/2)*(alpha**2 - 1)**6*(-3*alpha**2 + alpha + 4)/(sqrt(pi)*chi*sigma*(pi*(alpha - 1)**2*(alpha**2 - 1) - 4*(-3*alpha**2 + alpha + 4)**2)))

Eq(\kappa_o, -16*pi*Delta*T**6*chi*n**4*sigma*(alpha - 1)/(m**2*(pi**2*Delta**2*T**6*chi**2*n**4*sigma**2*(alpha - 1)**2/m**2 + pi*T**5*chi**2*n**4*sigma**2*(3*Delta**2*m + 15*T*alpha**2 - 4*T*alpha - 19*T)**2/(4*m**3))))

Eq(\kappa_o, -16*(alpha - 1)*(alpha**2 - 1)/(Delta*chi*sigma*(pi*(alpha - 1)**2*(alpha**2 - 1) - 4*(-3*alpha**2 + alpha + 4)**2)))

# Diffusion

In [11]:
N_parallel_parallel = sp.simplify(I.dot(I.c1, I.c1p) - I.dot(I.c1, I.c1))*vT**2
N_parallel_parallel_averaged = I.compute(N_parallel_parallel)
N_parallel_parallel_averaged = sp.factor(I.substitution_vt_T(I.dimensionalize(N_parallel_parallel_averaged)))
   

N_parallel_perp = sp.simplify(I.dot(I.c1perp, I.c1p) - I.dot(I.c1perp, I.c1))*vT**2
N_parallel_perp_averaged = I.compute(N_parallel_perp)
N_parallel_perp_averaged = sp.factor(I.substitution_vt_T(I.substitution_delta_star(I.dimensionalize(N_parallel_perp_averaged))))

display(sp.Eq(sp.Symbol("N_{\\parallel\\perp}", real=True), N_parallel_perp_averaged))
display(sp.Eq(sp.Symbol("N_{\\parallel\\perp}", real = True), I.substitution_Temp(N_parallel_perp_averaged)).simplify())
display(sp.Eq(sp.Symbol("N_{\\parallel\\parallel}", real=True), N_parallel_parallel_averaged)) 
display(sp.Eq(sp.Symbol("N_{\\parallel\\parallel}", real = True), I.substitution_Temp(N_parallel_parallel_averaged)).simplify())


Eq(N_{\parallel\perp}, pi*Delta*T*chi*n**2*sigma/m)

Eq(N_{\parallel\perp}, -pi*Delta**3*chi*n**2*sigma/(alpha**2 - 1))

Eq(N_{\parallel\parallel}, -2*sqrt(pi)*T**(3/2)*chi*n**2*sigma*(alpha + 1)/m**(3/2))

Eq(N_{\parallel\parallel}, -2*sqrt(pi)*Delta**3*chi*n**2*sigma*(-1/(alpha**2 - 1))**(3/2)*(alpha + 1))

In [12]:
Bo = 2*m*T/m*N_parallel_parallel_averaged/(N_parallel_perp_averaged**2 + N_parallel_parallel_averaged**2)
Boperp= 2*m*T/m*N_parallel_perp_averaged/(N_parallel_perp_averaged**2 + N_parallel_parallel_averaged**2)
D = -Bo*n*T/m
Do = Boperp*n*T/m
display(sp.Eq(sp.symbols('D'), D))
display(sp.Eq(sp.symbols('D'), I.substitution_Temp(D).simplify()))
display(sp.Eq(sp.symbols('D'), sp.limit(sp.limit(D, Delta, 0), alpha, 1)))
display(sp.Eq(sp.symbols('D_o'), Do))
display(sp.Eq(sp.symbols('D_o'), I.substitution_Temp(Do).simplify()))

Eq(D, 4*sqrt(pi)*T**(7/2)*chi*n**3*sigma*(alpha + 1)/(m**(5/2)*(pi**2*Delta**2*T**2*chi**2*n**4*sigma**2/m**2 + 4*pi*T**3*chi**2*n**4*sigma**2*(alpha + 1)**2/m**3)))

Eq(D, -4*Delta*m*(-1/(alpha**2 - 1))**(7/2)*(alpha + 1)*(alpha**2 - 1)**3/(sqrt(pi)*chi*n*sigma*(4*(alpha + 1)**2 - pi*(alpha**2 - 1))))

Eq(D, sqrt(T)*sqrt(m)/(2*sqrt(pi)*chi*n*sigma))

Eq(D_o, 2*pi*Delta*T**3*chi*n**3*sigma/(m**2*(pi**2*Delta**2*T**2*chi**2*n**4*sigma**2/m**2 + 4*pi*T**3*chi**2*n**4*sigma**2*(alpha + 1)**2/m**3)))

Eq(D_o, 2*Delta*m/(chi*n*sigma*(4*(alpha + 1)**2 - pi*(alpha**2 - 1))))

# Sonine

In [13]:
a2 = sp.symbols("a_2", real = True)
def S2(csq):
    return csq**2/2 - 2*csq + 1
delta_c2 = sp.simplify(I.norm2(I.c1p) - I.norm2(I.c1) + I.norm2(I.c2p) - I.norm2(I.c2))
delta_c4 = sp.simplify(I.norm2(I.c1p)**2 - I.norm2(I.c1)**2 + I.norm2(I.c2p)**2 - I.norm2(I.c2)**2)/2
to_integrate2 = delta_c2*(1 + a2*(S2(I.norm2(I.c1)) + S2(I.norm2(I.c2)))) # + O(a2^2)
to_integrate4 = delta_c4*(1 + a2*(S2(I.norm2(I.c1)) + S2(I.norm2(I.c2))))
delta_c2_averaged = I.compute(to_integrate2)
delta_c4_averaged = I.compute(to_integrate4)
delta_c2_averaged_sub = I.substitution_vt_T(I.substitution_delta_star(delta_c2_averaged))
delta_c4_averaged_sub = I.substitution_vt_T(I.substitution_delta_star(delta_c4_averaged))
display(sp.Eq(sp.Symbol(r"\langle{\delta c^2}\rangle"), delta_c2_averaged_sub))
display(sp.Eq(sp.Symbol(r"\langle{\delta c^4}\rangle"), delta_c4_averaged_sub))

Eq(\langle{\delta c^2}\rangle, sqrt(2)*sqrt(pi)*(-Delta**2*a_2*m/T + 16*Delta**2*m/T + 3*a_2*alpha**2 - 3*a_2 + 16*alpha**2 - 16)/16)

Eq(\langle{\delta c^4}\rangle, sqrt(2)*sqrt(pi)*(-Delta**4*a_2*m**2/T**2 + 16*Delta**4*m**2/T**2 + 6*Delta**2*a_2*alpha**2*m/T + 29*Delta**2*a_2*m/T + 32*Delta**2*alpha**2*m/T + 112*Delta**2*m/T + 30*a_2*alpha**4 + 147*a_2*alpha**2 - 32*a_2*alpha - 209*a_2 + 32*alpha**4 + 80*alpha**2 - 112)/64)

In [14]:
T_sonine = sp.solve(sp.Eq(delta_c2_averaged_sub, 0), T)[0]
T_sonine = sp.factor(T_sonine)
display(sp.Eq(sp.Symbol('T_{sonine}'), T_sonine))

# These are the two solutions for a2, not alpha.
solutions = sp.solve([sp.Eq(delta_c2_averaged, 0), sp.Eq(delta_c4_averaged, 0)], (a2, Delta_star))
sol_1 = solutions[0][0]
sol_2 = solutions[1][0]
sol = sol_1
if sol.limit(alpha, 1) != 0:
    sol = sol_2
sp.Eq(sp.symbols('a_2'), sp.simplify(sol))

Eq(T_{sonine}, Delta**2*m*(a_2 - 16)/((3*a_2 + 16)*(alpha - 1)*(alpha + 1)))

Eq(a_2, 16*(11*alpha**3 - 11*alpha**2 + 57*alpha + 4*sqrt(10*alpha**6 - 20*alpha**5 + 103*alpha**4 - 210*alpha**3 + 303*alpha**2 - 484*alpha + 314) - 73)/(39*alpha**3 - 39*alpha**2 + 273*alpha - 305))

# $\mu^1$ Boltzmann equation

In [15]:
if 0:
    import sympy as sp

    x, y, t = sp.symbols('x y t', real=True)

    m = sp.symbols('m', positive=True, real=True)
    vx, vy = sp.symbols('v_x v_y', real=True)  

    # --- hydrodynamic fields ---
    n  = sp.Function('n')(x, y, t)
    T  = sp.Function('T')(x, y, t)
    ux = sp.Function('u_x')(x, y, t)
    uy = sp.Function('u_y')(x, y, t)

    u = sp.Matrix([ux, uy])
    v = sp.Matrix([vx, vy])
    c = v - u
    c2 = (c.T*c)[0]

    def grad(f):
        return sp.Matrix([sp.diff(f, x), sp.diff(f, y)])

    def div(vec):
        return sp.diff(vec[0], x) + sp.diff(vec[1], y)

    grad_u = sp.Matrix([[sp.diff(ux, x), sp.diff(ux, y)],
                        [sp.diff(uy, x), sp.diff(uy, y)]])

    f0 = m*n/(2*sp.pi*T)*sp.exp(-m*c2/(2*T))

    p = n*T
    subs = {
        sp.diff(n, t):  -div(n*u),
        sp.diff(T, t):  -(u.T*grad(T))[0] - T*div(u),
        sp.diff(ux, t): -(u.T*grad(ux))[0] - (1/(m*n))*sp.diff(p, x),
        sp.diff(uy, t): -(u.T*grad(uy))[0] - (1/(m*n))*sp.diff(p, y),
    }

    Id = sp.eye(2)
    D = m*(c*c.T - (c2/2)*Id)                 
    A = (m*c2/2 - 2*T)*c                   

    D_colon_grad_u = (D*grad_u).trace()      # sum_{i,j} D_ij (\partial_i u_j)
    A_dot_gradlogT = (A.T*grad(sp.log(T)))[0]

    rhs = (1/T)*(D_colon_grad_u + A_dot_gradlogT)*f0

    lhs = (sp.diff(f0, t) + (v.T*grad(f0))[0]).subs(subs)  

    # Pretty replacements for display
    cx, cy = sp.symbols('c_x c_y', real=True)
    fo = sp.Symbol('f_0', real=True)
    subs_pretty = {
        vx - ux: cx,
        vy - uy: cy,
    }
    lhs_pretty = sp.simplify(lhs).subs(subs_pretty)
    rhs_pretty = sp.simplify(rhs).subs(subs_pretty)




    display(sp.Eq(sp.Symbol("LHS"), lhs_pretty))
    display(sp.Eq(sp.Symbol("RHS"), rhs_pretty))
    display(sp.Eq(sp.Symbol("LHS - RHS"), sp.simplify(lhs - rhs)))


